In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [ ]:
url = "http://www.gspns.rs/red-voznje/{}"

oblasti = {"gradski": 1, "prigradski": 2}
dani = {"radni dan" : 1, "subota" : 2, "nedelja" : 3}

df = pd.DataFrame(columns=["ID linije", "Lista polazaka"])

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
driver.get(url.format("gradski"))

In [ ]:
#menjanje dana
def promeni_dan(dan):
    try:
        driver.find_element(By.CSS_SELECTOR, '#dan > option:nth-child({})'.format(dan)).click()
    except:
        pass


In [ ]:
#otvori sve linije
def otvori_sve_linije():
    br_linija = len(driver.find_elements(By.CSS_SELECTOR, '#linija > option'))
    for linija in range(1, br_linija + 1):
        try:
            driver.find_element(By.CSS_SELECTOR, '#linija > option:nth-child({})'.format(linija)).click()
        except:
            pass
    try:
        driver.find_element(By.CSS_SELECTOR, '#rvform > button').click()
    except:
        pass

In [ ]:
#menjanje oblasti
def promeni_oblast(oblast):
    try:
        driver.find_element(By.CSS_SELECTOR, '#rv > option:nth-child({})'.format(oblast)).click()
    except:
        pass

In [ ]:
#svi polasci za dati dan i liniju
def get_polasci_oblast(dan, oblast):
    promeni_oblast(oblasti[oblast])
    time.sleep(0.5)
    promeni_dan(dan)
    time.sleep(0.5)
    otvori_sve_linije()
    time.sleep(0.5)
    br_linija = len(driver.find_elements(By.CSS_SELECTOR, '#linija > option'))
    for tabela in range(br_linija + 1):
        redni_br_polaska = 0
        polasci = []
        while(True):
            try:
                driver.find_element(By.CSS_SELECTOR, '#ispis-polazaka > table:nth-child({}) > tbody > tr:nth-child(2) > td:nth-child(1) > b:nth-child({})'.format(2 + 3 * tabela, 2 + redni_br_polaska)).click()
            except:
                break
            try:
                driver.find_element(By.CSS_SELECTOR, '#ispis-polazaka > table:nth-child({}) > tbody > tr:nth-child(2) > td:nth-child(1) > sup:nth-child({}) > font > span'.format(2 + 3 * tabela, 2 + redni_br_polaska+1)).click()
            except:
                break
            polazak = driver.find_element(By.CSS_SELECTOR, '#ispis-polazaka > table:nth-child({}) > tbody > tr:nth-child(2) > td:nth-child(1) > b:nth-child({})'.format(2 + 3 * tabela, 2 + redni_br_polaska)).text
            polasci.append(polazak)
            redni_br_polaska += 1
            time.sleep(0.5)
        df.append({"ID linije": tabela, "Lista polazaka": polasci}, ignore_index=True)


In [ ]:
def get_polasci(dan):
    get_polasci_oblast(dan, "gradski")
    get_polasci_oblast(dan, "prigradski")

In [ ]:
#svi polasci za sve dane i linije
def get_raspored():
    for dan in dani:
        get_polasci(dani[dan])

In [ ]:
#ispis-polazaka > table:nth-child(2) > tbody > tr:nth-child(2) > td:nth-child(1) > b:nth-child(2) prva tabela 1
#ispis-polazaka > table:nth-child(5) > tbody > tr:nth-child(2) > td:nth-child(1) > b:nth-child(2) druga tabela 1
#ispis-polazaka > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(1) > b:nth-child(2) treca tabela 1
#ispis-polazaka > table:nth-child(11) > tbody > tr:nth-child(2) > td:nth-child(1) > b:nth-child(2) cetvrta tabela 1

In [ ]:
get_raspored()